
# Скачиваем необходимые библиотеки и модели.

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip3 install pytesseract
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-rus
!pip3 install opencv-contrib-python --upgrade

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-3l8des28
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-3l8des28
     |████████████████████████████████| 64 kB 2.3 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369107 sha256=f2322b239e92a1da5ddb89eb21cdf8542a42e8f08b5b14e921a42d35024e4392
  Stored in directory: /tmp/pip-ephem-wheel-cache-0f14zgiu/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=b214a86a8bcd35a6e7fc59288b111d0a8ab65ef392ee2d812e9dc8c751fe9983
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built clip ftfy
  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=c1fd363e745c5a16a64f2d980e3e4cbfe3257cf5580feb1948224a897c1fa6c6
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/

In [ ]:
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from torchvision.models import detection
import torchvision
from tqdm import tqdm, tqdm_notebook
from PIL import Image
import matplotlib
import pytesseract
import cv2
import re
import clip

Здесь, возможно, другой путь необходим для тестового датасета.

In [ ]:
!unzip -q "drive/MyDrive/Colab Notebooks/Тестовые снимки.zip"

Это нужно для тестирования. Рисует предсказания детектора

In [ ]:
def draw_pic_with_rect(picture, boxes, labels, n=5):
    '''
        picture: torch.Tensor or np.array
        boxes: indexed object
        labels: indexed object
        draws a picture with detected objects
    '''
    boxes = boxes[:n]
    labels = labels[:n]
    if isinstance(picture, torch.Tensor):
        picture = (picture.detach().squeeze(0) * 256).permute(1, 2, 0).numpy()
    picture = picture.astype(dtype=np.int)

    fig, ax = plt.subplots(figsize = (15, 15))
    ax.imshow(picture)

    for box, lab in zip(boxes, labels):
        rect = matplotlib.patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                       linewidth=1, edgecolor='r', facecolor='none')
        ax.text(box[0], box[1], inst_classes[lab], fontsize = 12)
        ax.add_patch(rect)

    fig.show()

классы, которые выдаёт детектор

# Константы

In [ ]:
inst_classes = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

Вот эти константы нужны будут для модели CLIP.

In [ ]:
animals = [
            "an animal", 
            "not an animal",
            "a person",
            "a bench",
            "a car",
]

is_dog = [
            "a dog",
            "a cat",
            "a bird",
            "a bench",
]

interensting_indices = [
    inst_classes.index("bird"),
    inst_classes.index("cat"),
    inst_classes.index("dog"),
    inst_classes.index("horse"),
    inst_classes.index("sheep"),
    inst_classes.index("cow"),
    inst_classes.index("elephant"),
    inst_classes.index("bear"),
    inst_classes.index("zebra"),
    inst_classes.index("giraffe"),
    inst_classes.index("bench"),          
    inst_classes.index("person"),
]

Здесь мы каждой породе сопоставили длину её хвоста

In [ ]:
converter_tail = {
    "a dachshund": 1,
    "a borzoi" : 2,
    "a basset hound" : 2,
    "a spitz": 1,
    "a yorkshire terrier": 1,
    "a chihuahua": 1,
    "a german shepherd" : 2,
    "a labrador" : 2,
    "a husky" : 2,
    "a jack russell terrier" : 2,
    "a caucasian shepherd dog": 1,
    "a corgi": 1,
    "a bulldog": 1,
    "a pug": 1,
    "a poodle": 1,
    "a golden retriever" : 2,
    "a rottweiler": 1,
    "a beagle" : 2,
    "a cocker spaniel": 1,
    "a deutscher boxer": 1,
    "a pekingese": 1,
    "a sharpey": 1,
    "a bull terrier" : 2,
    "a doberman": 1,
    "a st. bernard" : 2,
    "a newfoundland" : 2,
    "a border collie" : 2,
    "a shiba inu": 1,
    "a dalmatian" : 2,
    "a samoyed" : 2,
    "a bichon frise": 1,
    "a toy terrier": 1,
    "a cavalier king charles spaniel": 2,
}

Токены для цвета собаки.

In [ ]:
converter_color = {
    "a black dog": 1,
    "a white dog": 2,
    "a brown dog": 3,
    "an orange dog": 3,
    "a chocolate dog": 3,
    "a liver dog": 3,
    "a red dog": 3,
    "a gold dog": 2,
    "a cream dog": 2,
    "a fawn dog": 2,
    "a blue dog": 3,
    "a gray dog": 2,
    "a grey dog": 2,
}

считаем на GPU, потому что так намного быстрее

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# Загружаем модели

In [ ]:
# детекция
model_detection = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained = True).to(device=device)

# CLIP
model, preprocess = clip.load("ViT-B/32", device=device)

# superresolution
superresolution = cv2.dnn_superres.DnnSuperResImpl_create()
path = "FSRCNN_x4.pb"
superresolution.readModel(path)
superresolution.setModel("fsrcnn",4)

# Класс для нахождения параметров собаки

In [ ]:
class DogParameters:
    '''
        Класс, который хранит в себе параметры собаки
    '''
    def __init__(self, 
                 filename, 
                 is_animal_there, 
                 is_it_a_dog, 
                 is_the_owner_there, 
                 color, 
                 breed,  
                 tail, 
                 address, 
                 cam_id,  
                 date,
                 box,):
        '''
            filename: имя фотографии
            is_animal_there: есть ли на этой фотографии животное
            is_it_a_dog: есть ли на этой фотографии собака
            is_the_owner_there: есть ли хозяин собаки на фотографии
            color: цвет собаки
            breed: порода собаки
            tail: длина хвоста собаки
            address: адрес камеры
            cam_id: id камеры
            date: дата
            box: координаты прямоугольника с собакой на фотографии
        '''
        
        self.res = {
            "filename": filename,
            "is_animal_there": is_animal_there,
            "is_it_a_dog": is_it_a_dog,
            "is_the_owner_there": is_the_owner_there,
            "color": color,
            "breed": breed,
            "tail": tail,
            "address": address,
            "cam_id": cam_id,
            "date": date,
            "box": box,
        }


In [ ]:
class FindLostAnimal():
    def __init__(self, 
                 model_detection, 
                 model_clip, 
                 preprocess,
                 model_superres,
                 interesting_indices,
                 animals, 
                 is_animal_index,
                 is_dog,
                 is_dog_index,
                 breeds,
                 colors,
                 converter_tail,
                 converter_color,
                 person_label=1,
                 device="cpu"
                 ):
        '''
            model_detection: модель для детекции 
            model_clip: модель CLIP
            preprocess: необходимые преобразования с картинкой, нужные CLIP
            model_superres: superresolution модель для улучшения качества 
                фотографии, отдаваемой пользователю
            interesting_indices: индексы у классов, которые возвращает детектор,
                в которых теоретически может быть собака
            animals: токены для проверки является ли животным
            is_animal_index: индекс животного в animals
            is_dog: токены для проверки является животное собакой
            is_dog_index: индекс собаки в is_dog
            breeds: токены для пород
            colors: токены для цветов
            converter_tail: по породе говорит длину хвоста
            converter_color: по цвету говорит цвет, необходимый пользователю (
                светлый, темный, разноцветный
            )
            person_label: индекс человека в inst_classes
            device: на чем считать
        '''
        
        # делаем инициализацию
        self.device = device
        self.model_detection = model_detection
        self.model_clip = model_clip
        self.preprocess_clip = preprocess
        self.model_superres = model_superres
        self.interesting_indices = interesting_indices
        self.tokens_animal = clip.tokenize(animals).to(self.device)
        self.tokens_color = clip.tokenize(colors).to(self.device)
        self.tokens_is_dog = clip.tokenize(is_dog).to(self.device)
        self.tokens_breed = clip.tokenize(breeds).to(self.device)
        self.is_animal_index = is_animal_index
        self.is_dog_index = is_dog_index
        self.breeds = breeds
        self.colors = colors
        self.converter_tail = converter_tail
        self.converter_color = converter_color
        self.person_label = person_label

        # переводим модели в режим тестирования
        self.model_detection.eval()
        self.model_clip.eval()

    def __get_cropped_image(self, picture: torch.Tensor, coordinates: list, expand_coeff=3.0):
        '''
            функция для обрезания фотографии с 
            каким-то расширяющим коэффициентом
        '''
        x = coordinates[0]
        y = coordinates[1]
        w = coordinates[2] - coordinates[0]
        h = coordinates[3] - coordinates[1]
        w_expanded = int(w * expand_coeff)
        h_expanded = int(h * expand_coeff)
        x_expanded = int(x - (w_expanded - w) / 2.0)
        y_expanded = int(y - (h_expanded - h) / 2.0)

        return torchvision.transforms.functional.crop(
            picture, 
            int(y_expanded), 
            int(x_expanded), 
            int(h_expanded), 
            int(w_expanded),
        )

    def __check_intersection(self, cr_1, cr_2):
        '''
            проверяют, пересекаются ли прямоугольники
        '''
        x1 = [cr_1[0], cr_1[2]]
        y1 = [cr_1[1], cr_1[3]]
        x2 = [cr_2[0], cr_2[2]]
        y2 = [cr_2[1], cr_2[3]]

        return not (max(x1) < min(x2) or 
                    min(x1) > max(x2) or 
                    max(y1) < min(y2) or 
                    min(y1) > max(y2))


    def __drop_uninformative_boxes(
                                    self, 
                                    boxes, 
                                    labels, 
                                    probabilities, 
                                    dog_label, 
                                    THRESHOLD_SIZE=25, 
                                    THRESHOLD_PROBABILITY=0.08
        ):
        '''
            В этой функции мы выбрасываем какие-то прямоугольники:
                - если собаки пересекаются
                - если детектор очень неуверен в том, что есть кто-то
        '''

        # плохие прямоугольники, которые мы потом выкинем
        bad_boxes = np.zeros(len(boxes))
        for i in range(len(boxes)):

            # если минимальная сторона квадратика маленькая или вероятность мала
            if min(boxes[i][2] - boxes[i][0], boxes[i][3] - boxes[i][1]) <= THRESHOLD_SIZE or \
            (probabilities[i] < THRESHOLD_PROBABILITY and labels[i] != dog_label):
                bad_boxes[i] = 1

        for i in range(len(boxes)):

            # если уже выкинули прямоугольник, то нет смысла что-то ещё с ним делать
            if bad_boxes[i] == 1:
                    continue
            for j in range(i + 1, len(boxes)):

                # если уже выкинули прямоугольник, 
                # то нет смысла что-то ещё с ним делать
                if bad_boxes[j] == 1:
                    continue

                # если пересекаются
                if self.__check_intersection(boxes[i], boxes[j]):

                    # если обе собаки, то оставляем с наибольшей площадью
                    if labels[i] == dog_label and labels[j] == dog_label:
                        # считаем площади
                        s1 = (boxes[i][2] - boxes[i][0]) * \
                        (boxes[i][3] - boxes[i][1])
                        s2 = (boxes[j][2] - boxes[j][0]) * \
                        (boxes[j][3] - boxes[j][1])

                        if s1 > s2:
                            bad_boxes[j] = 1
                        else:

                            # уже нет смысла дальше смотреть на i-ую собаку,
                            # так как мы её выкинем
                            bad_boxes[i] = 1
                            break

        result_boxes = []
        result_labels = []

        for i in range(len(bad_boxes)):
            if bad_boxes[i] == 0:
                result_boxes.append(boxes[i])
                result_labels.append(labels[i].cpu().item())

        return result_boxes, result_labels

    def __extract_text(self, path):
        '''
            получаем название улицы, id камеры, дату из картинки
        '''
        image_cv = cv2.imread(path)

        # обычно оно находится в левом верхнем углу + нужно ускорить
        # время работы, поэтому обрезаем картинку
        image_cv = image_cv[:image_cv.shape[0] // 10, :image_cv.shape[0] // 3, :]

        # расширяем, тогда pytesseract лучше работает
        image_cv = cv2.resize(
            image_cv, (image_cv.shape[1] * 3, image_cv.shape[0] * 3)
        )
        string = pytesseract.image_to_string(image_cv, lang='eng+rus')

        # дальше используем регулярные выражения, чтобы вытащить текст из строки
        id = re.findall(r"\s.*_\d.*\s", string, re.M)
        if len(id) > 0:
            id = id[0][1:-1]
        else:
            id = None
        date = re.findall(r'^.*\d\d:\d\d:\d\d$', string, re.M)
        if len(date) > 0:
            date = date[0]
        else:
            date = None
        address = re.findall(r'^.*\sдом.*$', string, re.M)
        if len(address) > 0:
            address = address[0]
        else:
            address = None
        return id, address, date

    def __has_owner(self, dog, persons, THRESHOLD=4):
        '''
            проверяет, есть ли у собаки хозяин методом, который
            мы описывали в документации
        '''
        dist = np.zeros(len(persons), dtype=np.float32)
        for i in range(len(persons)):
                delta_x = abs((persons[i][2] + persons[i][0]) / 2 -  
                                (dog[2] + dog[0]) / 2) / ((dog[2] - dog[0]))

                delta_y = abs((persons[i][3] + persons[i][1]) / 2 -  
                                (dog[3] + dog[1]) / 2) / (persons[i][3] - persons[i][1])

                dist[i] = max(delta_x, delta_y)
        return np.sum(dist < THRESHOLD) > 0

    def __is_an_animal(self, image) -> bool:
        '''
            проверяет, животное ли на фотографии
        '''
        logits_per_image, logits_per_text = model(image, self.tokens_animal)
        return logits_per_image[0].argmax().item() == self.is_animal_index

    def __is_a_dog(self, image) -> bool:
        '''
            проверяет, собака ли на фотографии
        '''
        logits_per_image, logits_per_text = model(image, self.tokens_is_dog)
        return logits_per_image[0].argmax().item() == self.is_dog_index

    def __get_breed_tail(self, image):
        '''
            вычисляем породу собаки на фотографии
        '''
        logits_per_image, logits_per_text = model(image, self.tokens_breed)
        breed = logits_per_image[0].argmax().item()
        return self.breeds[breed], self.converter_tail[self.breeds[breed]]

    def __get_color(self, image):
        '''
            вычисляем цвет собаки на фотографии
        '''
        logits_per_image, logits_per_text = model(image, self.tokens_color)
        return self.converter_color[self.colors[logits_per_image.argmax().item()]]

    def __get_features(self, image, box, persons, result_of_extract_text, path):
        '''
            Возвращает все признаки собаки
        '''
        is_the_owner_there = int(self.__has_owner(box, persons))
        color = self.__get_color(image)
        breed, tail = self.__get_breed_tail(image)
        cam_id, address, date = result_of_extract_text
        return DogParameters(
            os.path.basename(path), 
            1, 
            1, 
            is_the_owner_there, 
            color, 
            breed, 
            tail, 
            address, 
            cam_id, 
            date,
            box.tolist(),
        )

    def __not_a_dog(self, result_of_extract_text, path):
        '''
           Возвращает результат, если на картинке животное, но не собака 
        '''
        cam_id, address, date = result_of_extract_text
        return DogParameters(os.path.basename(path), 1, 0, 0, 0, 0, 0, address, cam_id, date, [])

    def __empty(self, result_of_extract_text, path):
        '''
            Возвращаем результат, если на картинке нет животного
        '''
        cam_id, address, date = result_of_extract_text
        return DogParameters(os.path.basename(path), 0, 0, 0, 0, 0, 0, address, cam_id, date, [])

    def __choose_answer(self, results: list) -> DogParameters:
        '''
            У нас может быть на картинке несколько результатов, 
            Выбираем только одну по такому параметру:
                - если есть собака, то возвращаем первую встретившуюся собаку, 
                так как детектор в ней наиболее уверен.
                - если собаки нет, то возвращаем первую картинку, так как
                в ней будет или животное или неживотное
        '''
        for result in results:
            if result.res["is_it_a_dog"] == 1:
                return result
        return results[0]

    def __call__(self, path, path_to_save=None):
        '''
            возвращает параметры собаки
            path: путь к картинке
            path_to_save: путь к директории, в которую сохранять
        '''
        image = Image.open(path)

        features = []
        # если картинка является png, то надо удалить последнюю карту
        picture_numpy = np.asarray(image)[:, :, :3]

        # преобразуем картинку в torch.Tensor
        picture_torch = torch.from_numpy(picture_numpy).\
        to(dtype=torch.float32, device=self.device).permute(2, 0, 1) / 256
        picture_torch.unsqueeze_(0)

        # получаем предсказания детектора
        detector_predictions = self.model_detection(picture_torch)[0]
        boxes = detector_predictions["boxes"]
        labels = detector_predictions["labels"]
        probabilities = detector_predictions["scores"]

        # отдельный случай для птички, потому что на фотография они все
        # очень маленькие, и дальше классификатор не может их распознать
        # как птичку, однако детектор делает это очень хорошо
        bird_here = (inst_classes.index("bird") in labels)

        # выбрасиваем плохие прямоугольники
        boxes, labels = self.__drop_uninformative_boxes(
            boxes, 
            labels, 
            probabilities,
            18,
        )

        # Получаем текст с фотографии
        result_of_extract_text = self.__extract_text(path)

        # Сохраняем всех людей, нужно для определения есть ли хозяин
        persons = [box for box, label in zip(boxes, labels) if label == self.person_label]

        # Главный цикл по прямоугольникам
        for box, label in zip(boxes, labels):

            # если индекс не интересный, то пропускаем
            if not label in self.interesting_indices:
                continue

            # нужно две картинки вырезать, потому что могут быть проблемы, когда
            # собака очень близко к человеку, то классификатор может распознать
            # человека, а не собаку
            cropped_image = self.__get_cropped_image(picture_torch, box)
            cropped_image_not_expanded = self.__get_cropped_image(
                picture_torch, box, 1.5
            )

            # не забываем преобразовать картинку для CLIP
            cropped_clip = self.preprocess_clip(
                torchvision.transforms.ToPILImage()(cropped_image[0])
            ).unsqueeze(0).to(self.device)
            cropped_clip_not_expanded = self.preprocess_clip(
                torchvision.transforms.ToPILImage()(
                    cropped_image_not_expanded[0]
                )
            ).unsqueeze(0).to(self.device)

            # елси не животное, то пропускаем
            if not self.__is_an_animal(cropped_clip_not_expanded):
                continue

            if self.__is_a_dog(cropped_clip):
            # если собака, то считаем параметры
                features.append(self.__get_features(
                    cropped_clip, 
                    box, 
                    persons, 
                    result_of_extract_text,
                    path
                ))
            else:
            # если не собака, то сохраняет это
                features.append(self.__not_a_dog(
                    result_of_extract_text,
                    path,
                ))

        # если в конце мы получили, что никого не обнаружили,
        # но есть птичка, то надо добавить, что есть животное
        if len(features) == 0 and bird_here:
            features.append(self.__not_a_dog(
                result_of_extract_text,
                path,
            ))

        # в противном случае нужно добавить, что никого нет
        if len(features) == 0:
            features.append(self.__empty(
                result_of_extract_text,
                path,
            ))

        # выбираем наиболее подходящий ответ
        result = self.__choose_answer(features)

        # если определён путь, то сохраняем картинку
        if not (path_to_save is None) and (result.res["is_it_a_dog"] == 1):
            image_to_save_torch = self.__get_cropped_image(picture_torch, result.res["box"], 2.0)
            image_to_save_numpy = image_to_save_torch[0].permute(1, 2, 0).cpu().detach().numpy() * 256

            # используем superresolution для улучшения качества
            image_to_save_numpy = self.model_superres.upsample(image_to_save_numpy)
            image_to_save_pil = Image.fromarray(image_to_save_numpy)
            image_to_save_pil.save(os.path.join(path_to_save, "cropped_" + result.res["filename"]))

        return result

# пример использования
find_lost_animal = FindLostAnimal(model_detection, 
               model, 
               preprocess, 
               superresolution, 
               interensting_indices, 
               animals, 
               0, 
               is_dog, 
               0, 
               list(converter_tail.keys()), 
               list(converter_color.keys()),
               converter_tail,
               converter_color,
               1,
               "cuda",
               )

result = find_lost_animal("/content/Тестовые снимки/не собаки/С39.jpg")
print(result.res)

{'filename': 'С39.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 0, 'color': 3, 'breed': 'a dachshund', 'tail': 1, 'address': 'ЮАО 1-й Нижний Михайловский проезд, дом |', 'cam_id': 'P¥N_hd_UAG_7270_3', 'date': '29 Е 2021, 06:49:46', 'box': [1099.9952392578125, 612.2385864257812, 1149.517333984375, 649.7343139648438]}


# Пример подсчёта всего тестового датасета

In [ ]:
!mkdir cropped

In [ ]:
empty_result = []
path_to_dir = "/content/Тестовые снимки/Пустые"
for image_name in tqdm(os.listdir(path_to_dir)):
    path_to_pic = os.path.join(path_to_dir, image_name)
    empty_result.append(
        find_lost_animal(path_to_pic, "/content/cropped").res
    )

100%|██████████| 48/48 [01:56<00:00,  2.43s/it]


In [ ]:
path_to_dir = "/content/Тестовые снимки/Только собаки"
only_dogs = []
for directory in os.listdir(path_to_dir):
    for image_name in tqdm(os.listdir(os.path.join(path_to_dir, directory))):
        path_to_pic = os.path.join(path_to_dir, directory, image_name)
        try:
            only_dogs.append(
                find_lost_animal(path_to_pic, "/content/cropped").res
            )
        except Exception:
            print("error while reading")

 14%|█▍        | 4/29 [00:11<01:08,  2.73s/it]

fucking fuck!!!


 66%|██████▌   | 19/29 [00:44<00:30,  3.04s/it]

fucking fuck!!!


 19%|█▉        | 7/37 [00:17<01:08,  2.29s/it]

...

  7%|▋         | 2/30 [00:04<01:04,  2.30s/it]

fucking fuck!!!


 83%|████████▎ | 25/30 [00:55<00:10,  2.01s/it]

fucking fuck!!!


100%|██████████| 31/31 [01:21<00:00,  2.63s/it]


In [ ]:
owner_and_dog = []
path_to_dir = "/content/Тестовые снимки/Хоязин и собака"
for image_name in tqdm(os.listdir(path_to_dir)):
    path_to_pic = os.path.join(path_to_dir, image_name)
    try:
        owner_and_dog.append(
            find_lost_animal(path_to_pic, "/content/cropped").res
        )
    except Exception:
        print("error while reading")

  9%|▉         | 6/67 [00:20<03:18,  3.26s/it]

fucking fuck!!!


 15%|█▍        | 10/67 [00:29<02:52,  3.02s/it]

fucking fuck!!!


 42%|████▏     | 28/67 [01:21<02:18,  3.55s/it]

...

 72%|███████▏  | 48/67 [02:19<01:04,  3.42s/it]

fucking fuck!!!


 93%|█████████▎| 62/67 [03:01<00:19,  3.88s/it]

fucking fuck!!!


100%|██████████| 67/67 [03:10<00:00,  2.84s/it]


In [ ]:
not_dogs = []
path_to_dir = "/content/Тестовые снимки/не собаки"
for image_name in tqdm(os.listdir(path_to_dir)):
    path_to_pic = os.path.join(path_to_dir, image_name)
    try:
        not_dogs.append(
            find_lost_animal(path_to_pic, "/content/cropped").res
        )
    except Exception:
        print("error while reading")

100%|██████████| 33/33 [01:22<00:00,  2.50s/it]


In [ ]:
!zip -r cropped.zip cropped

In [ ]:
order = ['filename', 'is_animal_there', 'is_it_a_dog', 'is_the_owner_there', 'color', 'tail', 'address', 'cam_id',]
result_array = []
for res in empty_result:
    result_array.append(
        [res[key] for key in order]
    )

for res in only_dogs:
    result_array.append(
        [res[key] for key in order]
    )

for res in owner_and_dog:
    result_array.append(
        [res[key] for key in order]
    )

for res in not_dogs:
    result_array.append(
        [res[key] for key in order]
    )

In [ ]:
result_csv = pd.DataFrame(result_array, columns=order)
result_csv.to_csv("result_table_with_breed.csv", index=False)